# Example 3: Fluorescence intensity and shape of phytoplankton cells

Images kindly provided by Irene Gallego and Anita Narwani.

In this example we are trying to i) detect phytoplankton cells in 1-channel images (black and white images), ii) measure their pixel-intensities (=fluorescence intensities) and their their shape, and iii) work towards building a classifier for taxonomic groups of phytoplankton. The first two objectives are a rather low hanging fruit, as they involve classic computer vision approaches that phenopype can readily supply. The latter goal is more longterm and involves experimenting with classic shape feature detection (e.g. [image moments](https://docs.opencv.org/3.4/d8/d23/classcv_1_1Moments.html)), and potentially also building up towards a deep learning pipeline usign the contours detecting with phenopype. I will update this example as work progresses.  

## Shape descriptors
With the recently (phenopype 1.0.5) introducted `shape_features` function you can measure a set of 43 rotation invariant features from collected contours. 

In [1]:



import phenopype as pp

image_path = "images/phytoplankton.jpg" 



In [4]:
pp.pype(image_path, name="v1", preset="object_detection_morph", dirpath="_temp/output/ex3")

dirpath defaulted to file directory - E:\git_repos\phenopype\tutorials\images
Directory to save files set at - E:\git_repos\phenopype\tutorials\images
Save folder _temp/output/ex3 does not exist - create?.y
Did not find "pype_config_v1.yaml" -  create at following location? (y/n):
_temp/output/ex3\pype_config_v1.yaml
y
pype config generated from preset1.
Created and saved new pype config "pype_config_v1.yaml" in folder _temp/output/ex3
_temp/output/ex3\pype_config_v1.yaml


------------+++ new pype iteration 2020:05:11 12:05:37 +++--------------


Nothing loaded.
SEGMENTATION
threshold
find_contours
VISUALIZATION
select_canvas
- invalid selection - defaulting to raw image
draw_contours
EXPORT
save_contours
- contours saved under _temp/output/ex3\contours_v1.csv.
AUTOSAVE
save_canvas
- canvas saved under _temp/output/ex3\canvas_v1.jpg.


------------+++ new pype iteration 2020:05:11 12:06:50 +++--------------


Nothing loaded.
PREPROCESSING
invert_image
SEGMENTATION
threshold
find_conto

SystemExit: 

TERMINATE (by user)